# Compute diffusion scores in Python

In [27]:
import networkx as nx
import numpy as np
import logging
import scipy as sp
from math import pi, sqrt
import sys

logger = logging.getLogger()
logger.setLevel(logging.DEBUG)
logging.debug("test")

import os 
dir_path = os.path.dirname(os.path.realpath('__file__'))

DEBUG:root:test


#### Import kernel functions from diffuPy

The kernel functions a imported from the package. Despite this the functions implementation are in this notebook (final _imp in the function name).

In [28]:
from diffupy.kernel import commute_time_kernel, p_step_kernel, inverse_cosine_kernel, diffusion_kernel, regularised_laplacian_kernel

from diffupy.miscellaneous import Matrix, LaplacianMatrix


### Import example graph

In [29]:
G = nx.read_gml(dir_path+'/04_unit_testing/_graph.gml', label='id')

### General functions

#### Labels mapping

In [30]:
def csv_labeled_matrix_to_nparray(path):
    # Import matrix from csv file and remove headers
    m = np.genfromtxt(path, delimiter=',')
    return np.array([[x for x in a if ~np.isnan(x)] for a in m[1:]])

In [31]:
def run_score_test(score_func, G, input_scores, test_output_scores, z = False):
    computed_output_scores = score_func(G, input_scores, z = z)

    logging.info(' %s  \n %s\n', 'Computed matrix', computed_output_scores)
    logging.info(' %s  \n %s\n', 'Test matrix', test_output_scores)
    # Assert rounded similarity (floating comma)
    assert np.allclose(computed_output_scores, test_output_scores)
    logging.info(' Test '+ score_func.__name__ +' passed')

#### Helpers

In [32]:
# In which format is the input? Tell apart vector, matrix or list of matrices
def which_format(x):
    # if is.numeric(x) and is.null(dim(x)): return "vector"
    if isinstance(x, isinstance(x[0], list)) or isinstance(x, isinstance(x[0], np.ndarray)): return "matrix"
    if isinstance(x, list): return "list"
    
    raise ValueError('Non-recognised input scores format, object of class:  %s', x.__class__.__name__)

#### Checks

In [33]:
# TO-DO
#.check_method
#.check_metric

# Labels matrix !!

# def check_scores(scores):
#     form = which_format(scores)
#     #scores_names = names(scores)
#     #if is.null(scores_names):
#     #    raise ValueError("Scores must be a named list, but supplied list contains no names.")

#     plyr::l_ply(
#         scores_names,
#         function(mat_name) {
#             mat <- scores[[mat_name]]

#             if (!is.numeric(mat) & !("dgCMatrix" %in% class(mat))) {
#                 stop(
#                     "The scores in background ",
#                     mat_name,
#                     " are not numeric!"
#                 )
#             }
#             if (any(is.na(mat))) {
#                 stop(
#                     "Scores input cannot contain NA! ",
#                     "But background ",
#                     mat_name,
#                     " does!")
#             }
#             if (is.null(rownames(mat)))
#                 stop(
#                     "The scores in background ",
#                     mat_name,
#                     " must have rownames ",
#                     "according to the scored nodes!"
#                 )
#             if (is.null(colnames(mat)))
#                 stop(
#                     "The scores in background ",
#                     mat_name,
#                     " must have colnames ",
#                     "to differentiate score sets!"
#                 )

#             std <- apply(mat, 2, stats::sd)
#             std_zero <- which(std == 0)
#             std_na <- which(is.na(std))

#             if (length(std_na))
#                 warning(
#                     "Standard deviation in background ",
#                     mat_name,
#                     " is NA in columns: ",
#                     paste(std_na, collapse = ",")
#                 )
#             if (length(std_zero))
#                 warning(
#                     "Standard deviation in background ",
#                     mat_name,
#                     " is 0 in columns: ",
#                     paste(std_zero, collapse = ",")
#                 )
#         }
#     )
#     invisible()
# }

## Diffuse scores

Diffuse scores on a network Function diffuse takes a network in networkx format and an initial state to score all the nodes in the network.

In [34]:
def calculate_scores(col_ind, scores, diff, const_mean, const_var):
    col_in = scores[:, col_ind]
    col_raw = diff[:, col_ind]

    s1 = np.sum(col_in)
    s2 = np.sum(col_in**2)

    # means and vars depend on first and second moments
    # of the input. This should be valid for non-binary
    # inputs as well
    score_means = const_mean*s1
    score_vars = const_var*(len(scores)*s2 - s1**2)
    
    return np.subtract(col_raw, score_means)/np.sqrt(score_vars)

def diffuse_raw (graph,
    scores,
    z = False,
    K = None,
    *argv):
    
    # TODO
    # sanity checks
    # .check_scores(scores)

    # Kernel matrix
    if K is None:
        # .check_graph(graph)
        logging.info('Kernel not supplied. Computing regularised Laplacian kernel ...')
        K = regularised_laplacian_kernel(graph, normalized = False)
        logging.info('Done')
    else:
        # .check_K(K)
        logging.info('Using supplied kernel matrix...')

    # Compute scores        

    # TODO: match indices, for now we assume indices match
    # for i, input_scores in enumerate(scores):
    #    for j, score in enumerate(input_scores):
    #       id_label_mapping(i)
    # input scores

    # TODO: Sparse
    # scores.mat <- methods::as(scores[[scores.name]], "sparseMatrix")

    n = len(scores)

    # raw scores
    diff = np.matmul(K.mat[:,:n], scores.mat())

    # Return base matrix if it is raw
    # Continue if we want z-scores
    if not z: 
        return diff

    # If we want z-scores, must compute rowmeans and rowmeans2
    row_sums = np.array([round(np.sum(row), 2) for row in K[:, :n]])
    row_sums_2 = np.array([np.sum(row) for row in K[:, :n]**2])

    # Constant terms over columns
    const_mean = row_sums/n
    const_var = np.subtract(n*row_sums_2, row_sums**2)/((n - 1)*(n**2))    
    
    return np.transpose([ans.append(np.array(calculate_scores(i, scores, diff, const_mean, const_var))) for i in range(len(diff[0]))])

### Raw scores test

In [35]:
run_score_test(diffuse_raw, G, csv_labeled_matrix_to_nparray(dir_path+'/scores_test/input_scores.csv'), csv_labeled_matrix_to_nparray(dir_path+'/scores_test/output_scores.csv'))



INFO:root:Kernel not supplied. Computing regularised Laplacian kernel ...
INFO:root:Done


AttributeError: 'numpy.ndarray' object has no attribute 'get_mat'

### z-scores test

In [ ]:
run_score_test(diffuse_raw, G, csv_labeled_matrix_to_nparray(dir_path+'/scores_test/input_scores.csv'), csv_labeled_matrix_to_nparray(dir_path+'/scores_test/output_z_scores.csv'), z = True)